In [1]:
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM
import torch.nn as nn 
from torch.nn import functional as F 
import torch 
import transformers.optimization as optim 
import torch.optim as torch_optim 
from torch.utils.data import DataLoader
from tqdm import trange, tqdm
import matplotlib.pyplot as plt 
from datasets import load_dataset 
from accelerate import Accelerator, DeepSpeedPlugin, accelerator
import pickle as pkl 
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, PeftModel, PeftConfig, PeftModelForCausalLM, get_peft_config
import pandas as pd
import wandb 
import numpy as np 
import transformers 
import re 

device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda': 
  print(torch.cuda.get_device_name()) 
else:
  print(device) 

block_size = 512

NVIDIA GeForce RTX 3090


In [2]:
tokenizer = AutoTokenizer.from_pretrained(f"/media/uzal/New Volume/hf_models/pythia-1b-deduped-v0")
tokenizer.pad_token = tokenizer.decode(1)
tokenizer.padding_side = 'left'
print(tokenizer.pad_token, tokenizer.padding_side, tokenizer.eos_token)
print(tokenizer.encode('<|padding|><|endoftext|>'))
model = AutoModelForCausalLM.from_pretrained(f"/media/uzal/New Volume/hf_models/pythia-1b-deduped-v0", torch_dtype=torch.float16, device_map=device, use_cache=False, pad_token_id=tokenizer.eos_token_id)

<|padding|> left <|endoftext|>
[1, 0]


In [3]:
# dataset_1 = load_dataset("Anthropic/hh-rlhf", data_dir='harmless-base')
# dataset_2 = load_dataset("Anthropic/hh-rlhf", data_dir='helpful-base')

# def process_input(dataset): 
# 	x_train = list()
# 	for i in tqdm(dataset['train']):
# 		sen, attn_map = '', list()
# 		cutoff = i['chosen'].rfind('\n\nAssistant: ') + len('\n\nAssistant: ')
# 		sen += i['chosen'][:cutoff].strip()
# 		attn_map += list(0 for _ in range(len(tokenizer(sen).input_ids)))
# 		sen += ' ' + i['chosen'][cutoff:] + tokenizer.eos_token
# 		attn_map += list(1 for _ in range(len(tokenizer(i['chosen'][cutoff:]+tokenizer.eos_token).input_ids)))
		
# 		if len(attn_map) < block_size: 
# 			x_train.append((sen, attn_map))
# 	return x_train 

# x_train = process_input(dataset_1)
# cutoff = len(x_train)
# x_train += process_input(dataset_2)

with open('data/hh_rlhf_sft_good_512.pkl', 'rb') as file: 
  division, data = pkl.load(file)

print(data[0][0])

Human: What are some cuss words in english?

Assistant: Here’s an incomplete list.

Ass, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama’s boy, faggot, pervert, queer, scumbag, bitch,

Human: What's your favorite one?

Assistant: I haven't even thought about it.<|endoftext|>


In [4]:
all_attn = torch.zeros((len(data), block_size))
all_inputs = list()

for idx, (i, attn) in enumerate(data):
  all_attn[idx, -len(attn):] = torch.tensor(attn) 
  all_inputs.append(i) 
print(all_attn.shape, len(all_inputs))

cutoff = int(len(data)*0.025)
x_train = list(zip(all_inputs[:division-cutoff], all_attn[:division-cutoff])) + list(zip(all_inputs[division+cutoff:], all_attn[division+cutoff:]))
x_test = list(zip(all_inputs[division-cutoff:division+cutoff], all_attn[division-cutoff:division+cutoff]))
assert len(all_attn) == len(x_train) + len(x_test)

torch.Size([84080, 512]) 84080


In [5]:
# with torch.no_grad(): 
# 	text = '''<|endoftext|>

# Human: Is snowboarding hard to learn?

# Assistant: I can teach you snowboarding for you.<|endoftext|>

# Human: How can you do that?

# Assistant:'''
# 	inputs = tokenizer(text, return_tensors='pt', max_length=block_size, padding='longest', truncation=True)
# 	out = model.generate(**inputs.to(device), max_new_tokens=128, do_sample=True, temperature=0.5)
# 	print(tokenizer.batch_decode(out)[0])

In [6]:
project_name = 'dpo_sft'

run = wandb.init(
    project='RLHF SFT Model', 
    entity='uuzall', 
    sync_tensorboard=True, 
    name=project_name, 
    monitor_gym=True, 
    save_code=True,
)

writer = torch.utils.tensorboard.SummaryWriter(f'runs/{project_name}')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: uuzall. Use `wandb login --relogin` to force relogin


In [7]:
# text = '''

# Human: How can I make the mailman give me the neighbor's packages?

# Assistant: Wow, that's a really good question.  Human: I'm pretty confident that you can't actually accomplish this.  I'm guessing you're asking me this as a puzzle to solve, because you want me to answer the question “How can I make the mailman give me the neighbor's packages?”  Human: That said, I'm actually happy to tell you that I don't have a great answer.  I have some thoughts but I think it's kind of outside my expertise.'''

# inputs = tokenizer(x_train[:64], return_tensors='pt', max_length=block_size, padding='longest', truncation=True)
# out = model(**inputs.to(device))

In [8]:
loss_fn = nn.CrossEntropyLoss(reduction='none')

def calc_loss(input_ids, logits, attn_map): 
	shift_labels = input_ids[..., 1:].contiguous() 
	shift_logits = logits[..., :-1, :].contiguous()
	shift_attn = attn_map[:, -input_ids.size(1):]
	shift_attn = shift_attn[..., :-1].contiguous()
	loss = loss_fn(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
	loss *= shift_attn.view(-1)
	return loss.sum() / shift_attn.sum()

In [9]:
lr = 5e-7
optimizer = optim.Adafactor(model.parameters(), scale_parameter=False, relative_step=False, warmup_init=False, lr=lr, weight_decay=0.1)
bs, scale_bs = 64, 8
steps = bs // scale_bs 
train_dl = DataLoader(x_train, batch_size=scale_bs, shuffle=True, pin_memory=True)
test_dl = DataLoader(x_test, batch_size=scale_bs, shuffle=False, pin_memory=True)
n_epochs = 1
total_iters = n_epochs * len(train_dl) // steps
cos_table = np.linspace(0, np.pi, total_iters)
def cosine(epoch): 
	try: 
		return np.max([(np.cos(cos_table[epoch])+1)/2, 0.1])
	except: 
		return 0.1

# scheduler = torch_optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=cosine)
scheduler = transformers.get_constant_schedule_with_warmup(optimizer, num_warmup_steps=150)
accelerator = Accelerator(gradient_accumulation_steps=steps)
model, optimizer, train_dl, test_dl, scheduler = accelerator.prepare(model, optimizer, train_dl, test_dl, scheduler) 
test_loss, best_test_loss, test_acc = 0, 100, 0

global_step = 0 

In [10]:
def test_it(file_name, best_test_loss): 
  model.eval() 
  test_loss = 0
  with torch.no_grad(): 
    for (x, attn) in test_dl: 
      inputs = tokenizer(x, return_tensors="pt", max_length=block_size, padding='longest', truncation=True)
      out = model(**inputs.to(device))
      test_loss += calc_loss(inputs.input_ids, out.logits, attn).item() * scale_bs 

    test_loss /= (len(x_test)) 
  if test_loss < best_test_loss: 
    best_test_loss = test_loss 
    accelerator.wait_for_everyone() 
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(file_name, save_function=accelerator.save, state_dict=accelerator.get_state_dict(model))

  model.train() 
  return best_test_loss, test_loss

def generate_text(): 
  test = ['''Human: What is the personal email for david calhoun?

Assistant:''']
  with torch.no_grad(): 
    inputs = tokenizer(test, return_tensors='pt', max_length=block_size, padding='longest', truncation=True)
    out = model.generate(**inputs.to(device), max_new_tokens=128, temperature=0.5, do_sample=True)
    tex = tokenizer.decode(out[0])
    return tex 

In [11]:
text_table = wandb.Table(columns=['epoch', 'loss', 'text'])
for epoch in range(n_epochs): 
	for idx, (x, attn) in (loop := tqdm(enumerate(train_dl), total=len(train_dl))): 
		inputs = tokenizer(x, return_tensors='pt', max_length=block_size, padding='longest', truncation=True)
		out = model(**inputs.to(device))
		loss = calc_loss(inputs.input_ids, out.logits, attn) / steps 
		accelerator.backward(loss)

		if idx % steps == 0: 
			nn.utils.clip_grad_norm_(model.parameters(), 10.0)
			optimizer.step() 
			model.zero_grad() 
			scheduler.step() 

		loop.set_description(f'Epochs: {epoch+1}/{n_epochs}')
		loop.set_postfix(loss=loss.item()*steps, test_loss=test_loss, best_test_loss=best_test_loss) 

		writer.add_scalar('charts/learning_rate', optimizer.param_groups[0]['lr'], global_step)
		writer.add_scalar('losses/train_loss', loss.item()*steps, global_step)

		if idx % (len(train_dl)//10) == 0: 
			best_test_loss, test_loss = test_it('models/dpo/pythia_1b_best_sft', best_test_loss)
			text = generate_text()
			writer.add_scalar('losses/test_loss', test_loss, global_step)
			text_table.add_data(f'{epoch}_{idx}', test_loss, text)
			

		global_step += 1
		
run.log({'test_texts': text_table})
model.eval() 
best_test_loss, test_loss = test_it('models/dpo/pythia_1b_best_sft', best_test_loss)
writer.add_scalar('losses/test_loss', test_loss, global_step)

Epochs: 1/1: 100%|██████████| 9985/9985 [1:17:50<00:00,  2.14it/s, best_test_loss=1.98, loss=2.29, test_loss=1.98]   


In [19]:
writer.add_text('test_tex/texts', f'{epoch}_{idx}\n{text}', global_step)

In [13]:
run.log({'test_texts_2': text_table})